In [1]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
# import osmnx as ox
import numpy as np
from itertools import product

# import sys
# sys.path.append('/Users/danielsuarez/Documents/Academic/Spring2025/SeniorDesign/CMOR492-DWS/')

from network_construction.network import source_treatment, get_Utown



In [2]:
G = get_Utown()
source_nodes, treatment_nodes = source_treatment(G, 40)  # <-- Specify # starting points for treatment node algorithm

networkx.classes.digraph.DiGraph

In [4]:
G.edges[(59156807, 59162550)]

{'osmid': 7942860,
 'name': 'Mc Gorkle Street',
 'highway': 'residential',
 'oneway': False,
 'reversed': True,
 'length': 41.891999999999996}

In [ ]:
paths = nx.all_pairs_shortest_path(G, weight='length')

In [9]:
p0 = nx.shortest_path(G, source=59110295, target=59110295, weight='length')
print(len(p0) - 1)
print(nx.path_weight(G, p0, weight='length'))

0
0


In [6]:
G.nodes[59110295] == G.nodes[59110295]

True

In [26]:
### MODEL PARAMETERS

Path = {}  # Set of shortest paths from each source node i to each treatment node j
NLinks = {}  # Number of edges in each path
L = {}  # Length of each path (distance)


for u, v in product(G.nodes, repeat=2):
    path = nx.shortest_path(G, source=u, target=v, weight='length')
    Path[u, v] = path
    NLinks[u, v] = len(path) - 1
    L[u, v] = nx.path_weight(G, path, weight='length')



# TODO: What if we just make pipe size continuous/linear


LE = {e: G.edges[e]['length'] for e in G.edges}  # Length of edge e
EL = {v: G.nodes[v]['elevation'] for v in G.nodes}  # Elevation of node v


D = [0.2, 0.25, 0.3, 0.35, 0.40, 0.45]  # Pipe diameters
CP = {0.05: 8.7, 0.06: 9.5, 0.08: 11,
                       0.1: 12.6, 0.15: 43.5, 0.2: 141,
                       0.25: 151, 0.3: 161, 0.35: 180,
                       0.4: 190, 0.45: 200}  # Cost per unit of pipe


SR = {}  # Production at node u
CAP = {}  # Treatment Capacity at node u if plant built


for node in source_nodes:
    G.nodes[node]['production'] = 0.05
    SR[node] = 0.05


total_flow = sum(SR.values())


for node in treatment_nodes:
    G.nodes[node]['capacity'] = 1000
    CAP[node] = 1000


Vmin = 0.6
Vmax = 3

CE = 25  # Cost of Excavation
CB = 6  # Cost of Bedding
TR = 44000  # Fixed Cost of Treatment Plant
TRFlow = 100  # Variable Cost of Treatment
PICost = 30

# PF = {'0.05': 8.7, '0.06': 9.5, '0.08': 11,
#                        '0.1': 12.6, '0.15': 43.5, '0.2': 141,
#                        '0.25': 151, '0.3': 161, '0.35': 180,
#                        '0.4': 190, '0.45': 200}  # Fixed Cost of Piping

CT = 1000000000  # Cost of trucking
M = 1e6

Smin = 0.01
Smax = 0.1
W = 0.5  # Buffer Width


In [27]:
m = gp.Model()

### DECISION VARIABLES

x = m.addVars(Path.keys(), vtype=GRB.BINARY, name='x') # Assign node u to treatment node v
y = m.addVars(G.nodes, vtype=GRB.BINARY, name='y')  # build treatment plant at node v
z = m.addVars(G.edges, vtype=GRB.BINARY, name='z')  # edge e used
d = m.addVars(G.edges, D, vtype=GRB.BINARY, name='d')  # Pipe size s at edge e

r = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb=0.0, name='r')  # flow handled at trucking at node u (recourse)

Q = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb=0.0, name='Q')  # Flow in Edge e

el = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, name='el')  # Elevation at node u

p = m.addVars(Path.keys(), vtype=GRB.CONTINUOUS, lb = 0.0, name='p') # Path flow

m.update()

In [28]:
### CONSTRAINTS

# NODE PRODUCTION MINUS RECOURSE
node_prod_rec = m.addConstrs((p[u, v] >= (SR[u] * x[u, v]) - r[u] for u, v in Path.keys()), name='node_prod_rec')

# TREATMENT CAPACITY
treat_cap = m.addConstrs((gp.quicksum(p[i, j] for i in source_nodes) <= CAP[j] * y[j] for j in treatment_nodes), name='treat_cap')

#  NODE ASSIGNMENT
node_assign = m.addConstrs((gp.quicksum(x[i, j] for j in treatment_nodes) >= 1 for i in source_nodes), name='node_assignment')

# PIPE SIZING
pipe_sizing = m.addConstrs((gp.quicksum(d[*e, s] for s in D) == z[e] for e in G.edges), name='pipe_sizing')  # ALWAYS BE SURE TO EXPAND e

# TODO: Go through this with John
# FLOW DEFINITION
def is_sublist(short_list, long_list):
    for i in range(len(long_list) - len(short_list) + 1):
        if long_list[i:i + len(short_list)] == short_list:
            return True
    return False

flow_def = m.addConstrs((Q[e] == gp.quicksum(p[i, j] for i, j in Path.keys() if is_sublist(list((e[0], e[1])),Path[i,j])) for e in G.edges), name='flow_def')


# MIN/MAX SLOPE
min_slope = m.addConstrs((el[e[0]] - el[e[1]] >= (LE[e] * Smin) - (M * (1 - z[e])) for e in G.edges), name='min_slope')
max_slope = m.addConstrs((el[e[0]] - el[e[1]] <= (LE[e] * Smax) + (M * (1 - z[e])) for e in G.edges), name='max_slope')

# FLOW VELOCITY LIMIT
flow_vel = m.addConstrs((Q[e] <= Vmax * gp.quicksum((np.pi / 8) * (s**2) * (d[*e, s]) for s in D) for e in G.edges), name='flow_vel')

# PIPES UNDERGROUND
underground = m.addConstrs((el[i] <= EL[i] for i in source_nodes), name='underground')
m.update()

In [29]:
# EDGE ACTIVATION

# TODO: Go through this with John 2
# EDGE ACTIVATION
ePath = {}  # Use this for Edge Activiation Constraint
for e, p in Path.items():
    ePath[e] = [(p[l - 1], p[l]) for l in range(1, len(p))]

edge_activate = m.addConstrs((gp.quicksum(z[e] for e in ePath[i, j]) >= NLinks[i, j] * x[i, j] for i, j in Path), name='edge_activate')

In [30]:
# ENVELOPES FOR MANNING

T = 11.9879
P = lambda LE, s: LE / (T * (s**(16/3)))
Qmax = lambda s: Vmax * ((np.pi / 8) * (s**2))


alpha = m.addVars(G.edges, D, lb=0, name='alpha')
beta = m.addVars(G.edges, D, lb=0, name='beta')


alpha_2 = m.addConstrs((alpha[*e, s] >= Q[e] + d[*e, s] * Qmax(s) - ( Qmax(s)) for e in G.edges for s in D), name='alpha_2')
alpha_3 = m.addConstrs((alpha[*e, s] <= Qmax(s) * d[*e, s] for e in G.edges for s in D), name='alpha_3')
alpha_4 = m.addConstrs((alpha[*e, s] <= Q[e] for e in G.edges for s in D), name='alpha_4')
alpha_5 = m.addConstrs((alpha[*e, s] <= Qmax(s) for e in G.edges for s in D), name='alpha_5')

beta_2 = m.addConstrs((beta[*e, s] >= (Qmax(s) * Q[e]) + (Qmax(s) * alpha[*e, s]) - (Qmax(s)**2) for e in G.edges for s in D), name='beta_2')
beta_3 = m.addConstrs((beta[*e, s] <= Qmax(s) * alpha[*e, s] for e in G.edges for s in D), name='beta_3')
beta_4 = m.addConstrs((beta[*e, s] <= Qmax(s) * Q[e] for e in G.edges for s in D), name='beta_4')

# manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= 0 for e in G.edges), name='manning_2')

# ADDED THE BIG M THING HERE BUT IDK IF IT COULD BE IMPROVED
manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= (1-z[e])*M for e in G.edges), name='manning_2')
m.update()

In [ ]:
# # MANNING'S EQUATION
#
# # manning = m.addConstrs(
# #     (LE[u, v, 0] * gp.quicksum(d_es[u, v, 0, s] * ((Q[u, v, 0]**2) / ((11.9879 * (s**(8/3)))**2)) for s in D)
# #      <= el[u] - el[v] for u, v, _ in G.edges),
# #     name="manning")
#

# JOHN REFORMULATION
# manning = m.addConstrs((LE[u, v, 0] * ((Q[u, v, 0] / (11.9879 * (s**2.6666667)))**2) <=
#                         el[u] - el[v] + (M * (1 - d_es[u, v, 0, s])) for u, v, _ in G.edges for s in D),
#                        name="manning")

# m.update()

In [31]:
### OBJECTIVE EPXR 1: TREATMENT COSTS

treat_cost = gp.LinExpr()
for j in treatment_nodes:
    treat_cost.addTerms(TR, y[j])
    for i in source_nodes:
        treat_cost.addTerms(TRFlow * SR[i], x[i, j])

# OBJECTIVE EXPR 2: EXCAVATION COSTS
excav_cost_f = lambda u, v: gp.QuadExpr(CE * (((EL[u] - el[u]) + (EL[v] - el[v])) / 2) * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s]) for s in D))

# OBJECTIVE EXPR 3: BEDDING COSTS
bed_cost_f = lambda u, v: gp.LinExpr(CB * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s]) for s in D))
# OBJECTIVE EXPR 4: PIPE COSTS
pipe_cost_f = lambda u, v: gp.LinExpr(LE[u, v] * gp.quicksum(CP[s] * d[u, v, s] for s in D))

excav_bed_cost = gp.quicksum(excav_cost_f(u, v) + bed_cost_f(u, v) + pipe_cost_f(u, v) for u, v in G.edges)

# OBJECTIVE EXPR 5: RECOURSE TRUCKING

rec_cost = gp.LinExpr()
for i in source_nodes:
    rec_cost.addTerms(CT, r[i])

m.setObjective(treat_cost + excav_bed_cost + rec_cost, GRB.MINIMIZE)

# m.setObjective(0, GRB.MINIMIZE)

m.update()
print(f"Model has {m.NumVars} variables and {m.NumConstrs} constraints.")

Model has 17974 variables and 32093 constraints.


In [32]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 32093 rows, 17974 columns and 180829 nonzeros
Model fingerprint: 0x7437315c
Model has 6054 quadratic objective terms
Variable types: 10680 continuous, 7294 integer (7294 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+06]
  Objective range  [5e+00, 1e+09]
  QObjective range [3e+02, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-03, 1e+06]
Presolve removed 16814 rows and 4886 columns
Presolve time: 1.30s
Presolved: 20697 rows, 18506 columns, 142461 nonzeros
Variable types: 11635 continuous, 6871 integer (6871 binary)
Found heuristic solution: objective 7.338846e+09
Found heuristic solution: objective -7.24648e+10
Found heuristic solution: objective -2.69652e+11

Deterministic concurrent LP op

In [33]:
for v in r:
    if r[v].X > 0:
        print(r[v].VarName, r[v].X)

r_59079200 0.05
r_59079983 0.027439657444652012
r_59079984 0.05
r_59080764 0.05
r_59080765 0.05
r_59080775 0.05
r_59081555 0.047058648087169884
r_59081564 0.0075217479059686105
r_59081605 0.05
r_59081615 0.05
r_59081625 0.05
r_59081682 0.05
r_59081782 0.0033521704859389365
r_59081790 0.05
r_59081840 0.05
r_59081845 0.05
r_59082151 0.05
r_59082376 0.05
r_59082984 0.05
r_59084971 0.05
r_59084975 0.05
r_59084990 0.02643805509807655
r_59090484 0.026438055098076556
r_59091067 0.04839105766356029
r_59091076 0.05
r_59094292 0.026438055098076542
r_59095026 0.0028761101961530915
r_59095700 0.04829571420301461
r_59095757 0.05
r_59095758 0.05
r_59096556 0.004834555477773124
r_59096570 0.04804155471837998
r_59096600 0.05
r_59096602 0.05
r_59097709 0.05
r_59101042 0.05
r_59101100 0.05
r_59101385 0.05
r_59101386 0.03640388704438444
r_59101390 0.05
r_59102119 0.05
r_59103172 0.02620002495316497
r_59103180 0.05
r_59104191 0.05
r_59105503 0.05
r_59105510 0.04027219819860369
r_59106124 0.05
r_59106145 0

In [92]:
m.computeIIS()

for c in m.getConstrs():
    if c.IISConstr:
        print(f"Constraint {c.ConstrName} is in the IIS")

for v in m.getVars():
    if v.IISLB > 0:
        print(f"Lower bound of {v.VarName} is in the IIS")
    elif v.IISUB > 0:
        print(f"Upper bound of {v.VarName} is in the IIS")


Constraint node_assign[9936649913] is in the IIS
Constraint min_slope[9936649913,59112341] is in the IIS
Constraint edge_activate[9936649913,59128065] is in the IIS
Constraint edge_activate[9936649913,59116291] is in the IIS
Constraint edge_activate[9936649913,9941438054] is in the IIS
Constraint edge_activate[9936649913,59123881] is in the IIS
Constraint edge_activate[9936649913,59117577] is in the IIS
Constraint edge_activate[9936649913,59101385] is in the IIS
Constraint edge_activate[9936649913,59091084] is in the IIS
Constraint edge_activate[9936649913,59115372] is in the IIS
Constraint edge_activate[9936649913,59153773] is in the IIS
Constraint edge_activate[9936649913,59079983] is in the IIS
Constraint edge_activate[9936649913,59146899] is in the IIS
Constraint edge_activate[9936649913,59164150] is in the IIS
Constraint edge_activate[9936649913,59118652] is in the IIS
Constraint edge_activate[9936649913,59097341] is in the IIS
Constraint edge_activate[9936649913,59081566] is in t